# 1. 셀레니움으로 오피넷의 최저가 주유데이터를 구마다 여러개의 엑셀 파일로 추출하여 HDFS에 저장하고, 
# 2. 모든 파일을 통합하여 PANDAS로 데이터프레임으로 만들고,
# 최저가 주유소 5곳과 비싼 주유소 5곳을 지도위에 표기하라.
# (셀레니움 운영체제 버전이 다름 주의)
인스타 그램 내일까지 세커녜정(아무거나)만들어오기

In [52]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime
from bs4 import BeautifulSoup as bs
from datetime import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [53]:
# # Chrome 옵션 설정
# options = Options()
# options.binary_location = "./chrome-win64/chrome.exe"  # chrome 경로 설정
# options.add_argument("--no--sandbox") # sandbox 설정
# options.add_argument("--disable-dev-shm-usage")  # 공유메모리 설정
# options.add_argument("--window-size=1920,1080") # (화면크기)해상도 설정, 반응형 웹 대비
# # options.add_argument("--disable-gpu") # GPU 비활성화 

# # ChromeDriver 설정
# service = Service("chromedriver.exe")
# driver = webdriver.Chrome(service=service, options=options)
# driver.get("https://www.opinet.co.kr/user/main/mainView.do")
# print(driver.title)

# # 요소찾기
# wait = WebDriverWait(driver, 10)
# hover_element = driver.find_element(By.CSS_SELECTOR, "#header > div > ul > li:nth-child(1) > a")
# menu_option = driver.find_element(By.CSS_SELECTOR, "#header > div > ul > li:nth-child(1) > ul > li:nth-child(1) > a > span")

# # 마우스 호버 액션
# actions = ActionChains(driver)
# actions.move_to_element(hover_element).perform() # 마우스 올린거 처럼

# # 약간의 시간을 기다린 후 클릭
# time.sleep(1)
# menu_option.click()
# time.sleep(5)

In [ ]:
# print(driver.page_source)
# frames = driver.find_elements(By.TAG_NAME, "iframe")
# print(f"iframe 갯수: {len(frames)}")
# for i, f in enumerate(frames):
#     print(f"[{i}] name: {f.get_attribute('name')}, id: {f.get_attribute('id')}")


In [66]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import UnexpectedAlertPresentException, NoAlertPresentException

# 🧱 다운로드 경로 설정
download_dir = os.path.abspath("downloads")
os.makedirs(download_dir, exist_ok=True)

# 🛠️ Chrome 옵션 설정 (다운로드 자동화)
options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
options.add_experimental_option("prefs", prefs)

# 🕒 다운로드 대기 함수 (.crdownload 감지)
def wait_for_download(download_path, timeout=15):
    seconds = 0
    while any(fname.endswith(".crdownload") for fname in os.listdir(download_path)):
        time.sleep(1)
        seconds += 1
        if seconds > timeout:
            raise Exception("⏳ 다운로드 대기 시간 초과")

# 🚀 드라이버 실행
driver = webdriver.Chrome(options=options)
driver.get("https://www.opinet.co.kr/user/main/mainView.do")
wait = WebDriverWait(driver, 10)

# 📂 메뉴 진입: 싼 주유소 찾기 > 경로별
hover = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#header > div > ul > li:nth-child(1) > a")))
webdriver.ActionChains(driver).move_to_element(hover).perform()
menu = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#header > div > ul > li:nth-child(1) > ul > li:nth-child(1) > a > span")))
menu.click()

# 🏙️ 시/도 선택: 부산광역시
wait.until(EC.presence_of_element_located((By.ID, "SIDO_NM0")))
Select(driver.find_element(By.ID, "SIDO_NM0")).select_by_index(2)
time.sleep(2)

# 📍 구 목록 수집
gugun_select = Select(driver.find_element(By.ID, "SIGUNGU_NM0"))
gugun_list = [opt.text for opt in gugun_select.options if opt.text not in ("전체", "시/군/구")]

print("📍 구 목록:", gugun_list)

# 🔁 각 구별 처리
for gugun in gugun_list:
    print(f"\n▶ {gugun} 처리 중...")

    # ✅ 구 선택
    Select(driver.find_element(By.ID, "SIGUNGU_NM0")).select_by_visible_text(gugun)
    time.sleep(2)

    try:
        # ✅ 엑셀 다운로드 버튼 클릭
        btn = wait.until(EC.element_to_be_clickable(
            (By.CSS_SELECTOR, "#templ_list0 > div:nth-child(7) > div > a > span")
        ))
        btn.click()

        # ✅ 다운로드 완료 대기
        wait_for_download(download_dir)

        # ✅ 가장 최근 .xls 파일 찾기
        files = [os.path.join(download_dir, f) for f in os.listdir(download_dir) if f.endswith(".xls")]
        latest_file = max(files, key=os.path.getctime)

        # ✅ 파일 이름 변경
        new_name = os.path.join(download_dir, f"{gugun}_주유소.xls")
        os.rename(latest_file, new_name)
        print(f"✅ 저장 완료 → {new_name}")

    except UnexpectedAlertPresentException:
        try:
            alert = driver.switch_to.alert
            print(f"⚠️ 경고창 발생: {alert.text} → 자동 닫음")
            alert.accept()
        except NoAlertPresentException:
            pass
        continue

    except Exception as e:
        print(f"❌ {gugun} 저장 실패: {e}")
        continue

# 🧹 마무리
driver.quit()
print("\n🎉 전체 구 다운로드 완료!")


📍 구 목록: ['강서구', '금정구', '기장군', '남구', '동구', '동래구', '부산진구', '북구', '사상구', '사하구', '서구', '수영구', '연제구', '영도구', '중구', '해운대구']

▶ 강서구 처리 중...
✅ 저장 완료 → D:\kgy\code\downloads\강서구_주유소.xls

▶ 금정구 처리 중...
✅ 저장 완료 → D:\kgy\code\downloads\금정구_주유소.xls

▶ 기장군 처리 중...
✅ 저장 완료 → D:\kgy\code\downloads\기장군_주유소.xls

▶ 남구 처리 중...
✅ 저장 완료 → D:\kgy\code\downloads\남구_주유소.xls

▶ 동구 처리 중...
✅ 저장 완료 → D:\kgy\code\downloads\동구_주유소.xls

▶ 동래구 처리 중...
✅ 저장 완료 → D:\kgy\code\downloads\동래구_주유소.xls

▶ 부산진구 처리 중...
✅ 저장 완료 → D:\kgy\code\downloads\부산진구_주유소.xls

▶ 북구 처리 중...
✅ 저장 완료 → D:\kgy\code\downloads\북구_주유소.xls

▶ 사상구 처리 중...
✅ 저장 완료 → D:\kgy\code\downloads\사상구_주유소.xls

▶ 사하구 처리 중...
✅ 저장 완료 → D:\kgy\code\downloads\사하구_주유소.xls

▶ 서구 처리 중...
✅ 저장 완료 → D:\kgy\code\downloads\서구_주유소.xls

▶ 수영구 처리 중...
✅ 저장 완료 → D:\kgy\code\downloads\수영구_주유소.xls

▶ 연제구 처리 중...
✅ 저장 완료 → D:\kgy\code\downloads\연제구_주유소.xls

▶ 영도구 처리 중...
✅ 저장 완료 → D:\kgy\code\downloads\영도구_주유소.xls

▶ 중구 처리 중...
✅ 저장 완료 → D:\kgy\code\downloads\중구_주유소.xls

▶ 해운

In [ ]:
# pip install openpyxl
# pip install xlrd

In [70]:
import pandas as pd
import os
from glob import glob

# 📂 다운로드 경로
download_dir = os.path.abspath("downloads")

# 🔍 .xls 파일 목록
xls_files = glob(os.path.join(download_dir, "*_주유소.xls"))

# 🧾 데이터 병합 리스트
merged_data = []

for file in xls_files:
    try:
        df = pd.read_excel(file)
        gu_name = os.path.basename(file).split("_")[0]  # 파일명에서 구이름 추출
        df.insert(0, "구", gu_name)  # 첫 컬럼에 구 정보 삽입
        merged_data.append(df)
    except Exception as e:
        print(f"⚠️ {file} 읽기 실패: {e}")

# 🧱 전체 병합
if merged_data:
    result_df = pd.concat(merged_data, ignore_index=True)
    output_path = os.path.join(download_dir, "부산_주유소_통합.xlsx")
    result_df.to_excel(output_path, index=False)
    print(f"\n✅ 병합 완료: {output_path}")
else:
    print("❌ 병합할 데이터가 없습니다.")



✅ 병합 완료: D:\kgy\code\downloads\부산_주유소_통합.xlsx


In [59]:
import os
import pandas as pd
from glob import glob

# 다운로드 폴더 절대 경로
download_dir = os.path.abspath("downloads")
os.makedirs(download_dir, exist_ok=True)

# .xls 파일 수집
xls_files = glob(os.path.join(download_dir, "*_주유소.xls"))

print("🧾 찾은 엑셀 파일들:")
print(f"총 개수: {len(xls_files)}")
for f in xls_files:
    print("-", os.path.basename(f))

# 통합
all_data = []
for file in xls_files:
    try:
        df = pd.read_excel(file, engine="xlrd")
        gugun = os.path.basename(file).split("_")[0]
        df['구'] = gugun
        all_data.append(df)
    except Exception as e:
        print(f"❌ {file} 읽기 실패: {e}")

# 저장
if all_data:
    merged_df = pd.concat(all_data, ignore_index=True)
    output_path = os.path.join(download_dir, "부산_주유소_통합.xlsx")
    merged_df.to_excel(output_path, index=False)
    print(f"\n📁 저장 완료: {output_path}")
else:
    print("❌ 읽을 수 있는 파일이 없습니다.")


🧾 찾은 엑셀 파일들:
총 개수: 15
- 금정구_주유소.xls
- 기장군_주유소.xls
- 남구_주유소.xls
- 동구_주유소.xls
- 동래구_주유소.xls
- 부산진구_주유소.xls
- 북구_주유소.xls
- 사상구_주유소.xls
- 사하구_주유소.xls
- 서구_주유소.xls
- 수영구_주유소.xls
- 연제구_주유소.xls
- 영도구_주유소.xls
- 중구_주유소.xls
- 해운대구_주유소.xls

📁 저장 완료: D:\kgy\code\downloads\부산_주유소_통합.xlsx


In [75]:
import pandas as pd
import os
from glob import glob

download_dir = os.path.abspath("downloads")
xls_files = glob(os.path.join(download_dir, "*_주유소.xls"))

columns = ["지역", "상호", "주소", "상표", "전화번호", "셀프여부", "고급휘발유", "휘발유", "경유", "실내등유"]
merged_data = []

for file in xls_files:
    try:
        df = pd.read_excel(file, header=None)

        # "단위 : 원/리터" 있는 줄 제거
        df = df[~df.apply(lambda row: row.astype(str).str.contains("단위").any(), axis=1)]

        # 실제 데이터 시작점 찾기 (보통 열 이름이 포함된 줄)
        start_idx = df[df.apply(lambda row: row.astype(str).str.contains("상호").any(), axis=1)].index
        if len(start_idx) == 0:
            print(f"❌ 컬럼명 줄을 찾을 수 없음: {file}")
            continue

        start_idx = start_idx[0]

        # 열 이름 및 본문 추출
        df.columns = columns
        df = df.loc[start_idx + 1:]
        df = df.dropna(how="all")  # 모두 비어있는 행 제거

        merged_data.append(df)

    except Exception as e:
        print(f"⚠️ 파일 오류 - {file}: {e}")

# 병합 후 저장
if merged_data:
    result = pd.concat(merged_data, ignore_index=True)
    output_file = os.path.join(download_dir, "부산_주유소_통합.xlsx")
    result.to_excel(output_file, index=False)
    print(f"\n✅ 저장 완료: {output_file}")
else:
    print("❌ 병합할 데이터가 없습니다.")



✅ 저장 완료: D:\kgy\code\downloads\부산_주유소_통합.xlsx


In [78]:
import pandas as pd
import os

# 📂 통합된 엑셀 파일 경로
file_path = os.path.join("downloads", "부산_주유소_통합.xlsx")

# 📖 데이터 로드
df = pd.read_excel(file_path, dtype=str)

# 🔢 휘발유 가격을 숫자로 변환 (숫자가 아닌 '-' 또는 NaN 제거)
df = df[df["휘발유"].str.isnumeric()]
df["휘발유"] = df["휘발유"].astype(int)

# 🔻 최저가 TOP 5
cheapest = df.sort_values(by="휘발유", ascending=True).head(5)

# 🔺 최고가 TOP 1
most_expensive = df.sort_values(by="휘발유", ascending=False).head(5)

# 📊 출력
print("⛽ 최저가 주유소 TOP 5:")
print(cheapest[["지역", "상호", "주소", "휘발유"]], "\n")

print("💸 가장 비싼 주유소:")
print(most_expensive[["지역", "상호", "주소", "휘발유"]])

⛽ 최저가 주유소 TOP 5:
        지역                 상호                     주소   휘발유
162  부산광역시  광신석유(주)직영 충렬대로주유소         부산 동래구 충렬대로 95  1489
163  부산광역시              대일주유소        부산 동래구 충렬대로 124  1499
164  부산광역시       ㈜OS에너지 동호주유소         부산 동래구 반송로 245  1499
166  부산광역시       동방석유(주)푸른주유소  부산 동래구 충렬대로 121 (온천동)  1499
165  부산광역시              명장주유소   부산 동래구 반송로 303 (명장동)  1499 

💸 가장 비싼 주유소:
        지역               상호                      주소   휘발유
161  부산광역시      ㈜삼현석유 천일주유소    부산 동구 중앙대로 168 (초량동)  1998
64   부산광역시  광신석유(주)직영 명지주유소       부산 강서구 르노삼성대로 372  1899
292  부산광역시       피치스주유소 송도점           부산 서구 감천로 227  1888
291  부산광역시    ㈜한국페트로 스타2주유소           부산 서구 원양로 260  1868
328  부산광역시   (주)나부코직영 우원주유소  부산 영도구 태종로 181 (봉래동4가)  1848


In [1]:
import folium
import pandas as pd

# 🛠️ 데이터 (네가 제공한 주유소 정보 복사해서 DataFrame으로 구성)
data = {
    "주소": [
        "부산 동래구 충렬대로 95",
        "부산 동래구 충렬대로 124",
        "부산 동래구 반송로 245",
        "부산 동래구 충렬대로 121 (온천동)",
        "부산 동래구 반송로 303 (명장동)",
        "부산 동구 중앙대로 168 (초량동)",
        "부산 강서구 르노삼성대로 372",
        "부산 서구 감천로 227",
        "부산 서구 원양로 260",
        "부산 영도구 태종로 181 (봉래동4가)"
    ],
    "휘발유": [1489, 1499, 1499, 1499, 1499, 1998, 1899, 1888, 1868, 1848],
    "위도": [35.2074394, 35.2058571, 35.2026025, 35.2064321, 35.2067787,
           35.1123058, 35.0893564, 35.0821804, 35.0742783, 35.0912922],
    "경도": [129.0715907, 129.0738774, 129.0989642, 129.0739197, 129.1027076,
           129.0387887, 128.8975986, 129.0125945, 129.0064934, 129.0493362]
}

df = pd.DataFrame(data)

# 🗺️ 지도 생성 (중심 좌표는 동래구 기준)
m = folium.Map(location=[35.2, 129.07], zoom_start=12)

# 📍 마커 추가
for i, row in df.iterrows():
    popup_text = f"{row['주소']}<br>휘발유: {row['휘발유']}원"
    folium.Marker(
        location=[row["위도"], row["경도"]],
        popup=popup_text,
        tooltip=row["휘발유"],
        icon=folium.Icon(color="green" if row["휘발유"] <= 1500 else "red", icon="oil", prefix="fa")
    ).add_to(m)

# 💾 HTML로 저장
m.save("부산_주유소_지도.html")
print("✅ 지도 생성 완료! → 부산_주유소_지도.html 파일을 브라우저에서 열어보세요.")

m

✅ 지도 생성 완료! → 부산_주유소_지도.html 파일을 브라우저에서 열어보세요.


In [81]:
# pip install geopy

Note: you may need to restart the kernel to use updated packages.
